<a href="https://colab.research.google.com/github/melvinlkl/urban-succotash/blob/main/GCP_Workshop_Func_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create An Agent with Gemini's Function Calling**

Credits: https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/creating_an_agent_with_function_calling.ipynb

In [ ]:
! pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
import vertexai

PROJECT_ID = "project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

assert PROJECT_ID != "your-project-id", "Please input your project id"
assert LOCATION != "your-location", "Please input your project location"

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import requests

from vertexai.generative_models import (
    GenerativeModel,
    FunctionDeclaration,
    Tool,
    Part,
)

In [ ]:
# create an agent
agent = GenerativeModel(
    model_name="gemini-1.5-flash"
)

# start a chat session
chat = agent.start_chat()

# ask the agent about exchange rate
prompt = "what's the current exchange rate of SGD to KRW?"
response = chat.send_message(prompt)

# display the response
response.candidates[0].content.parts[0]

text: "I do not have access to real-time information, including live currency exchange rates. \n\nTo get the most up-to-date exchange rate for SGD to KRW, I recommend checking a reliable online currency converter like:\n\n* **Google Finance:** Just search \"SGD to KRW\" on Google.\n* **XE.com:** A popular website for currency information.\n* **Yahoo Finance:** Another reputable source for financial data.\n\nRemember that exchange rates fluctuate constantly, so the rate you see at one moment may be different just a few minutes later. \n"

In [ ]:
# define a function to get exchange rate
def get_fx_rate(base_currency: str, target_currency: str):
    """
    Fetches the current exchange rate between two currencies.

    Args:
        base_currency: The base currency (e.g., "USD").
        target_currency: The target currency (e.g., "SGD").

    Returns:
        The exchange rate information as a json response,
        or None if the rate could not be fetched.
    """

    url = f"https://hexarate.paikama.co/api/rates/latest/{base_currency}?target={target_currency}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()


# test the function
get_fx_rate("SGD", "KRW")

{'status_code': 200,
 'data': {'base': 'SGD',
  'target': 'KRW',
  'mid': 1072.44,
  'unit': 1,
  'timestamp': '2024-12-25T08:58:13.528Z'}}

In [ ]:
# initialize function declaration
fd_get_fx_rate = FunctionDeclaration(
    name = "get_fx_rate",
    description = "Fetches the current exchange rate between two currencies.",
    parameters = {
        "type" : "object",
        "properties" : {
            "base_currency": {
                "type": "string",
                "description": "The base currency (e.g., 'USD')"
            },
            "target_currency": {
                "type": "string",
                "description": "The target currency (e.g., 'SGD')"
            },
        },
    }
)

In [ ]:
# create a tool with the function declaration
currency_tool = Tool(
    function_declarations=[
        fd_get_fx_rate
    ]
)

In [ ]:
# create an agent with the currency tool
agent = GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[currency_tool],
)

# start a chat session
chat = agent.start_chat()

# ask the agent about exchange rate
response = chat.send_message(prompt)

# display the response
response.candidates[0].content.parts[0]

function_call {
  name: "get_fx_rate"
  args {
    fields {
      key: "target_currency"
      value {
        string_value: "KRW"
      }
    }
    fields {
      key: "base_currency"
      value {
        string_value: "SGD"
      }
    }
  }
}

In [ ]:
response.candidates[0].content.parts[0].function_call

name: "get_fx_rate"
args {
  fields {
    key: "target_currency"
    value {
      string_value: "KRW"
    }
  }
  fields {
    key: "base_currency"
    value {
      string_value: "SGD"
    }
  }
}

In [ ]:
# assign function name and arguments as provided in the function call response
func_name = response.candidates[0].content.parts[0].function_call.name
func_args = response.candidates[0].content.parts[0].function_call.args

# call the function to get the exchange rate information
fx_rate_json = globals()[func_name](**func_args)
fx_rate_json

{'status_code': 200,
 'data': {'base': 'SGD',
  'target': 'KRW',
  'mid': 1072.44,
  'unit': 1,
  'timestamp': '2024-12-25T08:58:13.528Z'}}

In [ ]:
# provide function's response to the chat instance
response = chat.send_message(
    Part.from_function_response(
        name=func_name,
        response={
            "content": fx_rate_json,
        },
    ),
)

response.text

'The current exchange rate of SGD to KRW is 1072.44 KRW per SGD. \n'